In [1]:
import torch
import glob
import pytorch_lightning as pl
from ClimatExML.loader import ClimatExMLData
from ClimatExML.wgan_gp import SuperResolutionWGANGP
from lightning.pytorch.tuner import Tuner

# torch.set_float32_matmul_precision('medium')
torch.set_float32_matmul_precision('medium')
torch.cuda.empty_cache()

In [2]:
data_glob = {
    "lr_test": [
        glob.glob("/home/nannau/data/era5/test/U10/*.pt"),
        glob.glob("/home/nannau/data/era5/test/V10/*.pt"),
        glob.glob("/home/nannau/data/era5/test/ps/*.pt")
    ],
    "lr_train": [
        glob.glob("/home/nannau/data/era5/train/U10/*.pt"),
        glob.glob("/home/nannau/data/era5/train/V10/*pt"),
        glob.glob("/home/nannau/data/era5/train/ps/*.pt")
    ],
    "lr_train": [
        glob.glob("/home/nannau/data/era5/train/U10/*.pt"),
        glob.glob("/home/nannau/data/era5/train/V10/*.pt"),
        glob.glob("/home/nannau/data/era5/train/ps/*.pt")
    ],
    "hr_test": [
        glob.glob("/home/nannau/data/wrf/test/U10/*.pt"),
        glob.glob("/home/nannau/data/wrf/test/V10/*.pt"),
    ],
    "hr_train": [
        glob.glob("/home/nannau/data/wrf/train/U10/*.pt"),
        glob.glob("/home/nannau/data/wrf/train/V10/*.pt"),
    ]
}

In [3]:
from lightning.pytorch.loggers import MLFlowLogger

mlf_logger = MLFlowLogger(experiment_name="lightning_logs", tracking_uri="file:./ml-runs")
# trainer = Trainer(logger=mlf_logger)

In [4]:
srmodel = SuperResolutionWGANGP(data_glob=data_glob)
trainer = pl.Trainer(precision="16-mixed", accelerator="gpu", max_epochs=1, logger=mlf_logger, log_every_n_steps=50)#fast_dev_run=True)
# tuner = Tuner(trainer)

# Auto-scale batch size by growing it exponentially (default)
# tuner.scale_batch_size(srmodel, max_trials=1, mode="power")

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(srmodel)

Traceback (most recent call last):
  File "/home/nannau/miniconda3/envs/xesmf_env/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 290, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/nannau/miniconda3/envs/xesmf_env/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 383, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/nannau/miniconda3/envs/xesmf_env/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1083, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/nannau/miniconda3/envs/xesmf_env/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1076, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/nannau/miniconda3/envs/xesmf_env/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 214, in read_yaml
    raise MissingConfigExcepti

Epoch 0:  19%|█████████████████▏                                                                         | 1315/6983 [30:53<2:13:07,  1.41s/it, v_num=55af, MAE loss=0.982, MSE loss=1.630, wasserstein=2514.0]